# Job Description Data Cleaning Pipeline

## 1. Data Exploration and Initial Load
Load and explore the raw job description dataset

In [20]:
# Import basic libraries for data processing
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from tqdm import tqdm

In [21]:
# Raw dataset
jd_df = pd.read_csv("../0_raw_dataset/job_description.csv")

## 2. Clean Job Location Data
### 2.1 Remove Invalid Location Entries
Remove entries starting with "Contact" that contain no useful location information

In [22]:
location_delete = jd_df["location"].astype(str).str.startswith("Contact")
jd_df = jd_df[~location_delete]

### 2.2 Filter Location by Length
Keep only locations between 5 and 50 characters to remove incomplete or malformed entries

In [23]:
# jd_df[jd_df["location"].str.len() > 50][["job_title", "location"]]
jd_df = jd_df[jd_df["location"].str.len().between(5, 50)]

### 2.3 Extract Standardized Location Format
Use regex to extract "City, STATE" format from location strings

In [24]:
jd_df = jd_df.copy()


def clean_location(text):
    text = str(text).strip()
    match = re.search(r"[A-Za-z\s]+,\s*[A-Z]{2}$", text)
    return match.group(0) if match else None


jd_df["location_cleaned"] = jd_df["location"].apply(clean_location)

In [25]:
jd_df = jd_df.dropna(subset=["location_cleaned"]).copy()
keep_columns = ["job_title", "job_description", "location_cleaned"]
jd_df = jd_df[keep_columns]

## 3. Clean Job Title Data
### 3.1 Extract Job Title from Pattern
Extract the actual job title before "Job in [Location]" pattern

In [26]:
jd_df["job_title_cleaned"] = jd_df["job_title"].str.split(" Job in ").str[0]

In [27]:
def clean_job_title(title):
    if pd.isna(title):
        return None

    title = str(title).strip()

    # Remove "Job in {location}" pattern
    title = re.split(r"\s+Job in\s+", title, flags=re.IGNORECASE)[0]

    # Remove extra whitespace
    title = re.sub(r"\s+", " ", title)

    # Remove trailing dashes or slashes
    title = re.sub(r"[-/\s]+$", "", title)

    # Standardize case (optional - Title Case)
    # title = title.title()

    return title.strip()


jd_df["job_title_cleaned"] = jd_df["job_title"].apply(clean_job_title)

### 3.2 Remove Missing and Incomplete Titles
Drop rows with missing titles or titles shorter than 3 characters

In [28]:
# Check for titles without "Job in" pattern
no_pattern = jd_df[~jd_df["job_title"].str.contains("Job in", na=False)]
print(f"Titles without 'Job in' pattern: {len(no_pattern)}")

# Check for very long titles (might indicate messy data)
long_titles = jd_df[jd_df["job_title_cleaned"].str.len() > 100]
print(f"Titles longer than 100 chars: {len(long_titles)}")

# Remove rows with missing titles
jd_df = jd_df.dropna(subset=["job_title_cleaned"])

# Remove very short titles (likely incomplete)
jd_df = jd_df[jd_df["job_title_cleaned"].str.len() >= 3]

Titles without 'Job in' pattern: 785
Titles longer than 100 chars: 4


### 3.3 Standardize Similar Job Titles
Map variations of job titles to standardized forms (e.g., "Software Developer" → "Software Engineer")

In [29]:
# Map similar titles together
title_mapping = {
    "Software Engineer": ["software developer", "programmer", "dev engineer"],
    "Machine Learning Engineer": ["ml engineer", "machine learning"],
    "Project Manager": ["program manager"],
    "Data Analyst": ["data analyst"],
    "Resident Nurse": ["registered nurse"]}


def standardize_title(title):
    title_lower = title.lower()
    for standard, variations in title_mapping.items():
        if any(var in title_lower for var in variations + [standard]):
            return standard
    return title


jd_df["job_title_standardized"] = jd_df["job_title_cleaned"].apply(standardize_title)

### 3.4 Remove Invalid Job Descriptions
Filter out job descriptions containing specific chaotic patterns or spam content

In [30]:
# Check if job_description contains the phrase
pattern = (
    r"Please apply only if you are qualified\.|"
    r"\$K-\$K RESTAURANT MANAGER DALLAS! \$K-\$K KITCHEN MANAGER ARLINGTON! NEW STORES ACROSS DFW!"
)
contains_phrase = jd_df["job_description"].str.contains(
    pattern,
    case=False,  # case-insensitive
    na=False,  # treat NaN as False
    regex=True,
)

# Remove those rows
jd_df = jd_df[~contains_phrase]

In [31]:
# drop the unneeded title columns using the correct argument
jd_df = jd_df.drop(columns=["job_title", "job_title_cleaned"])
# rename the standardized title column to 'job_title'
jd_df.rename(columns={"job_title_standardized": "job_title"}, inplace=True)

## 4. Clean Job Description Content

### 4.1 Remove Invalid Descriptions
Filter out descriptions with placeholder text or spam patterns

In [32]:
df = jd_df.copy()

df = df[df["job_description"] != "Please apply only if you are qualified."]
# print(len(df))

### 4.2 Remove Duplicate Job Descriptions
Drop rows with duplicate job descriptions while preserving unique entries

In [33]:
# there are many duplicate value
dupes = df[df["job_description"].duplicated(keep=False)]
same_rows = dupes.groupby("job_description").filter(
    lambda g: g.drop(columns=["job_description"]).nunique().sum() == 0
)

# print(f"Same job description but different companies, locations, etc., after checking: {len(same_rows)}")

# drop them
duplicates = df["job_description"].value_counts()
num_duplicate_rows = duplicates[duplicates > 1].sum()
# print("Total number of duplicate job_description lines:", num_duplicate_rows)
# number not too much, drop them
df = df[~df["job_description"].duplicated(keep=False)]
# print(len(df))

### 4.3 Extract Key Sections: Duties, Requirements, and Education

#### 4.3.1 Pattern Analysis
Analyze keyword patterns to identify what percentage of jobs contain each section

In [34]:
# check key features in job description only using keywords

# === Responsibilities / Duties  ===
duties_pattern = (
    r"(?i)\b("
    r"responsibilit(?:y|ies)|"
    r"duties|"
    r"tasks|"
    r"key responsibilities|"
    r"primary responsibilities|"
    r"main duties|"
    r"essential duties|"
    r"job duties|"
    r"core responsibilities|"
    r"what you['’]ll do|"
    r"what you will do|"
    r"your role|"
    r"role overview|"
    r"day[- ]to[- ]day|"
    r"main responsibilities|"
    r"responsibilities|"
    r"key deliverables|"
    r"accountabilities|"
    r"scope of work|"
    r"what this role does"
    r")\b"
)


duties_mask = df["job_description"].str.contains(duties_pattern, case=False, na=False)


# === Requirements / Qualifications ===
req_pattern = (
    r"(?i)\b("
    r"requirements?|"
    r"qualifications?|"
    r"skills and experience|"
    r"required skills?|"
    r"preferred skills?|"
    r"experience required|"
    r"experience and education|"
    r"knowledge, skills|"
    r"competencies?|"
    r"core competencies?|"
    r"what you['’]ll need|"
    r"what you need|"
    r"what we expect|"
    r"what we['’]re looking for|"
    r"who you are|"
    r"what you bring|"
    r"your profile|"
    r"about you|"
    r"ideal candidate|"
    r"candidate profile|"
    r"person specification|"
    r"key attributes|"
    r"traits we['’]re seeking|"
    r"essential criteria|"
    r"selection criteria|"
    r"minimum qualifications?|"
    r"preferred qualifications?|"
    r"desired qualifications?|"
    r"education and experience|"
    r"education requirements?|"
    r"background required"
    r")\b"
)


req_mask = df["job_description"].str.contains(req_pattern, case=False, na=False)

# === Education / Degree / Certification  ===
edu_pattern = (
    r"(?i)\b("
    r"education|educational background|"
    r"degree[s]?|bachelor'?s|master'?s|ph\.?d|doctorate|mba|major|"
    r"college degree|university degree|high school diploma|ged|institution|"
    r"associate'?s degree|advanced degree|graduate degree|undergraduate degree|"
    r"certification[s]?|certified|license|required license|"
    r"credential[s]?|training required|academic background|academic requirements"
    r")\b"
)
edu_mask = df["job_description"].str.contains(edu_pattern, case=False, na=False)

total = len(df)
both_count = (edu_mask & req_mask).sum()
all_three_count = (duties_mask & req_mask & edu_mask).sum()

print(f"Total rows: {total}")
print(f"Both (Duties+Requirements): {both_count} ({both_count/total:.2%})")
print(f"All three present: {all_three_count} ({all_three_count/total:.2%})")

/var/folders/7c/t23ybbrs56z25_14648nh99w0000gn/T/ipykernel_76352/1567829725.py:30: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  duties_mask = df["job_description"].str.contains(duties_pattern, case=False, na=False)
/var/folders/7c/t23ybbrs56z25_14648nh99w0000gn/T/ipykernel_76352/1567829725.py:71: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  req_mask = df["job_description"].str.contains(req_pattern, case=False, na=False)
/var/folders/7c/t23ybbrs56z25_14648nh99w0000gn/T/ipykernel_76352/1567829725.py:84: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  edu_mask = df["job_description"].str.contains(edu_pattern, case=False, na=False)


Total rows: 5283
Both (Duties+Requirements): 3145 (59.53%)
All three present: 2537 (48.02%)


#### 4.3.2 Section Extraction - Version 4.5 (Keyword-Based)
Use keyword pattern matching to extract duties, requirements, and education sections

In [35]:
# ==========================================
# POSITIVE KEYWORDS
# ==========================================
DUTIES_POS = [
    r"responsibilit",
    r"duties",
    r"tasks",
    r"scope of work",
    r"deliverables",
    r"manage",
    r"support",
    r"operate",
    r"coordinate",
    r"lead",
    r"overs.*",
    r"execute",
    r"perform",
    r"work closely",
]

REQ_POS = [
    r"requirement",
    r"qualification",
    r"required",
    r"preferred",
    r"skills",
    r"experience",
    r"must have",
    r"ability to",
    r"proficiency",
    r"knowledge of",
    r"strong .* skills",
    r"familiarity",
    r"background in",
]

EDU_POS = [
    r"bachelor",
    r"master",
    r"ph\.?d",
    r"degree",
    r"school",
    r"diploma",
    r"ged",
    r"certification",
    r"certificate",
    r"license",
    r"major in",
]

# ==========================================
# NEGATIVE FILTER FOR REQUIREMENT
# ==========================================
REQ_NEG = [
    r"\boverview\b",
    r"\babout\b",
    r"\bcompany\b",
    r"\bmission\b",
    r"\bvision\b",
    r"\bvalue\b",
    r"\bwhy join\b",
    r"\bbenefit\b",
    r"\bwho we are\b",
    r"\bwhat we do\b",
    r"\bsummary\b",
]


def is_requirement_sentence(s):
    if not match_any(REQ_POS, s):
        return False
    if match_any(REQ_NEG, s):
        return False
    # avoid sentences that are too long (like full company descriptions)
    if len(s) > 220:
        return False
    return True


# ==========================================
# NEGATIVE FILTER FOR EDUCATION
# ==========================================
EDU_NEG = [
    r"duties",
    r"responsibilit",
    r"tasks",
    r"construction",
    r"supervis",
    r"manage",
    r"operations",
    r"project",
    r"customer",
    r"service",
    r"support",
    r"engineer",
    r"analysis",
    r"pipeline",
    r"maintenance",
]


def basic_clean(text):
    if pd.isna(text):
        return ""
    text = re.sub(r"<[^>]+>", " ", str(text))
    return re.sub(r"\s+", " ", text).strip()


def split_sentences(text):
    return [s.strip() for s in re.split(r"[\.!?;\n]+", text) if len(s.strip()) > 0]


def match_any(patterns, text):
    return any(re.search(p, text, re.IGNORECASE) for p in patterns)


def no_match(patterns, text):
    return not any(re.search(p, text, re.IGNORECASE) for p in patterns)


# ==========================================
# Strict EDU filter
# ==========================================
def is_education_sentence(s):

    # must contain positive EDU keyword
    if not match_any(EDU_POS, s):
        return False

    # must NOT contain typical duties/req keywords
    if match_any(DUTIES_POS, s):
        return False
    if match_any(REQ_POS, s):
        return False

    # maximum sentence length
    if len(s) > 200:
        return False

    return True


# ==========================================
# CORE extractor
# ==========================================
def extract_sections_v45(text):
    text = basic_clean(text)
    sentences = split_sentences(text)

    duties, reqs, edu = [], [], []

    for s in sentences:

        # education first (优先级更高)
        if is_education_sentence(s):
            edu.append(s)
            continue

        # requirements
        if is_requirement_sentence(s):
            reqs.append(s)
            continue

        # duties
        if match_any(DUTIES_POS, s):
            duties.append(s)
            continue

    return {
        "duties": "\n".join(duties),
        "requirements": "\n".join(reqs),
        "education": "\n".join(edu),
    }


# ==========================================
# DF WRAPPER
# ==========================================
def clean_jd_dataframe(df, col="job_description"):
    tqdm.pandas(desc="Extract JD V4.5 (Better Duties/Req/Edu)")
    result = df[col].progress_apply(extract_sections_v45)

    df["jd_duties"] = result.apply(lambda x: x["duties"])
    df["jd_requirements"] = result.apply(lambda x: x["requirements"])
    df["jd_education"] = result.apply(lambda x: x["education"])
    return df

In [36]:
df1 = df.copy()

df_clean = clean_jd_dataframe(df1)

Extract JD V4.5 (Better Duties/Req/Edu): 100%|██████████| 5283/5283 [00:04<00:00, 1252.34it/s]


In [37]:
cols = ["jd_duties", "jd_requirements", "jd_education"]

df_filtered = df_clean[df_clean[cols].apply(lambda x: x.str.len() >= 5).all(axis=1)]
print(len(df_filtered))
df_filtered[cols].head(5)
df_filtered[cols].to_excel("filtered_jd_sections1.xlsx", index=False)

813


#### 4.3.3 Section Extraction - Version 8.4 (SBERT + Hybrid Rules)
Use semantic similarity with SBERT embeddings combined with keyword rules for improved extraction

In [40]:
# 1. Load SBERT model
from sentence_transformers import SentenceTransformer
from sentence_transformers import util


model = SentenceTransformer("all-MiniLM-L6-v2")

# Semantic prototypes
DUTIES_PROTOTYPE = "responsibilities, duties, tasks the employee will perform"
REQ_PROTOTYPE = (
    "required skills, qualifications, licenses, certifications, work experience"
)
EDU_PROTOTYPE = "educational requirements, degree in, bachelors, masters, phd, diploma"

proto_embs = model.encode(
    [DUTIES_PROTOTYPE, REQ_PROTOTYPE, EDU_PROTOTYPE], convert_to_tensor=True
)
duties_proto, req_proto, edu_proto = proto_embs


# 2. Utility functions


def basic_clean(text):
    if pd.isna(text):
        return ""
    text = str(text)
    text = re.sub(r"<[^>]+>", " ", text)
    text = re.sub(r"\s+", " ", text)
    return text.strip()


def split_sentences(text):
    # Fix broken lines e.g. “Bachelor’s degree (B”
    text = text.replace("B .", "B.")
    text = re.sub(r"(?<=\bB)\s(?=[A-Z])", "", text)

    # sentence split
    sents = re.split(r"[\.!?;\n]+", text)
    return [s.strip() for s in sents if len(s.strip()) > 2]


def cosine_sim(a, b):
    return float(util.cos_sim(a, b).cpu().numpy())


def match_any(patterns, s):
    return any(re.search(p, s, re.IGNORECASE) for p in patterns)


def no_match(patterns, s):
    return not match_any(patterns, s)


# 3. Keyword dictionaries

# Strong education expressions (very strict)
EDU_STRONG = [
    r"bachelor.?s?\s+degree",
    r"bachelor\s+degree",  # Bachelor's degree / Bachelor degree
    r"master.?s?\s+degree",  # Master's degree
    r"High school" r"ph\.?d",
    r"doctorate",  # PhD / Doctorate
    r"degree in",
    r"major in",  # degree in X / major in X
    r"\bbs\b",
    r"\bba\b",
    r"\bms\b",
    r"\bma\b",  # BS / BA / MS / MA （
    r"b\.s\.",
    r"b\.a\.",
    r"m\.s\.",
    r"m\.a\.",
    r"graduate (student|level|program)",  # graduate-level / graduate student
]


# Basic degree keywords (weak)
EDU_CORE = [r"bachelor", r"master", r"ph\.?d", r"degree", r"diploma", r"ged"]

# Requirements keywords
REQ_POS = [
    r"requirement",
    r"qualification",
    r"required",
    r"skills",
    r"experience",
    r"proficiency",
    r"license",
    r"licensure",
    r"certification",
    r"certified",
    r"ms office",
    r"excel",
    r"powerpoint",
    r"word",
    r"outlook",
]

# Duties keywords
DUTIES_POS = [
    r"responsibilit",
    r"duties",
    r"tasks",
    r"deliverables",
    r"manage",
    r"support",
    r"coordinate",
    r"lead",
    r"execute",
]

# Company overview blocks (should NEVER be duties or requirements)
COMPANY_NEG = [
    r"company information",
    r"atos is",
    r"leader in",
    r"annual revenue",
    r"employees in",
    r"global",
    r"worldwide",
    r"training programs",
    r"olympic",
    r"paralympic",
    r"eeo",
    r"equal opportunity",
    r"our mission",
    r"our values",
    r"company culture",
    r"www\.",
    r"company website",
    r"vision",
    r"brand",
]

# Duties negative filters
DUTIES_NEG = COMPANY_NEG + [
    r"benefit",
    r"salary",
    r"compensation",
    r"401k",
    r"health",
    r"dental",
    r"insurance",
    r"pto",
    r"vacation",
    r"training program",
    r"development program",
]

# Requirements negative filters
REQ_NEG = COMPANY_NEG + [
    r"benefit",
    r"compensation",
    r"bonus",
    r"401k",
    r"pto",
    r"insurance",
    r"vacation",
    r"training program",
    r"career",
    r"growth",
    r"promotion",
]

# Education negative filters
EDU_NEG = COMPANY_NEG + [
    r"benefit",
    r"salary",
    r"insurance",
    r"pto",
    r"vacation",
    r"responsibilit",
    r"duties",
    r"tasks",
    r"operations",
    r"project",
    r"manage",
    r"customer",
    r"ms office",
    r"excel",
    r"powerpoint",
    r"word",
    r"outlook",
]


# 4. Sentence classifier
def classify_sentence(s, emb):
    duty_sim = cosine_sim(emb, duties_proto)
    req_sim = cosine_sim(emb, req_proto)
    edu_sim = cosine_sim(emb, edu_proto)

    # ----------- Education (Strong FIRST) -----------
    if match_any(EDU_STRONG, s) and no_match(EDU_NEG, s):
        return "edu"

    # ----------- Education (Weak → downgrade to requirements) -----------
    if match_any(EDU_CORE, s) and no_match(EDU_NEG, s):
        # If weak degree match and sentence also contains requirements stuff → requirements
        if match_any(REQ_POS, s):
            return "requirements"
        # Else education
        return "edu"

    # ----------- Duties keyword -----------
    if match_any(DUTIES_POS, s) and no_match(DUTIES_NEG, s):
        return "duties"

    # ----------- Duties semantic advantage -----------
    if duty_sim > req_sim + 0.05:
        return "duties"

    # ----------- Requirements -----------
    if (match_any(REQ_POS, s) or req_sim >= 0.38) and no_match(REQ_NEG, s):
        return "requirements"

    return None


# 5. Extractor
def extract_sections_v84(text):
    text = basic_clean(text)
    sents = split_sentences(text)

    if not sents:
        return {"duties": "", "requirements": "", "education": ""}

    sent_embs = model.encode(sents, convert_to_tensor=True)

    duties_s, req_s, edu_s = [], [], []

    for i, s in enumerate(sents):
        emb = sent_embs[i]
        cat = classify_sentence(s, emb)

        if cat == "duties":
            duties_s.append(s)
        elif cat == "requirements":
            req_s.append(s)
        elif cat == "edu":
            edu_s.append(s)

    return {
        "duties": "\n".join(duties_s),
        "requirements": "\n".join(req_s),
        "education": "\n".join(edu_s),
    }


# 6. DataFrame wrapper
def clean_jd_dataframe_v84(df, col="job_description"):
    tqdm.pandas(desc="Extract JD v8.4 (SBERT + Hybrid Rules)")
    result = df[col].progress_apply(extract_sections_v84)

    df = df.copy()
    df["jd_duties"] = result.apply(lambda x: x["duties"])
    df["jd_requirements"] = result.apply(lambda x: x["requirements"])
    df["jd_education"] = result.apply(lambda x: x["education"])
    return df

In [41]:
df2 = df.copy()

df_v8 = clean_jd_dataframe_v84(df2, col="job_description")

Extract JD v8.4 (SBERT + Hybrid Rules):   0%|          | 0/5283 [00:00<?, ?it/s]/var/folders/7c/t23ybbrs56z25_14648nh99w0000gn/T/ipykernel_76352/1985578339.py:44: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return float(util.cos_sim(a, b).cpu().numpy())
Extract JD v8.4 (SBERT + Hybrid Rules): 100%|██████████| 5283/5283 [10:08<00:00,  8.69it/s]


In [42]:
df_v8.head(5)

,job_description,location_cleaned,job_title,jd_duties,jd_requirements,jd_education
3,Why Join Altec? If you’re considering a career...,"Dixon, CA",Engineer - Quality,Our Company was founded based upon values that...,• EIT registration or ability to obtain regist...,
4,Position ID# 76162 # Positions 1 State CT C...,"Camphill, PA",Shift Supervisor - Part-Time,The primary purpose of this position is to pro...,"• In accordance with state law, candidates mus...",
5,Job Description Job #: 720298Apex Systems has...,"Charlottesville, VA",Construction PM - Charlottesville,This is a long-term rolling contract supportin...,,
9,"Insituform Technologies, LLC, an Aegion compan...","Chesterfield, MO",Video Data Management /Transportation Technician,"Insituform's businesses consist of sewer, drin...",Listed below are representative requirements f...,
12,Junior ProofreaderOur client's in- house creat...,"Boston, MA",Junior Proofreader,This person is responsible for checking proofs...,"Accuracy, attention to detail, and strong writ...",


In [44]:
cols = ["jd_duties", "jd_requirements", "jd_education"]

df_filtered2 = df_v8[df_v8[cols].apply(lambda x: x.str.len() > 5).all(axis=1)]
print(len(df_filtered2))
df_filtered2[cols].head(5)
df_filtered2.to_excel("filtered_jd_sections2.xlsx", index=False)
df_filtered2.to_excel("../5_checking_accuracy/jobdescription.xlsx", index=False)


1231


In [ ]:
print(
    jd_df[
        jd_df["job_description"].str.contains(
            "Extensive hand-on experience with Microsoft Active Directory", na=False
        )
    ]["job_description"]
)
print(jd_df.loc[1761, "job_description"])